In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../data/life_expectancy_data_cleaned.csv')
df.head()

## Build OLS model

In [ ]:
y = df.life_expectancy
X = df.drop(columns='life_expectancy')

In [ ]:
model=sm.OLS(y,add_constant(X))

In [ ]:
model_fit=model.fit()

In [ ]:
model_fit.summary()

## P-Hacking and model improvement

In [ ]:
# Defining a function to drop the column and return the new model

def phacking(column,X,y):
    
    if column:
        X=X.drop(column,axis=1)
    
    model=sm.OLS(y,add_constant(X)).fit()
    
    display(model.summary())
    
    return X

In [ ]:
dropped_cols = []

In [ ]:
dropped_cols.append('alcohol')
dropped_cols[-1]

In [ ]:
X=phacking(dropped_cols[-1],X,y)

In [ ]:
"""
Summary of the model:

The pvalue of F-statistic is under 0.05 so we have small chances of being wrong 
if we assume the parameters are not equals to 0 at the same time.

All our parameters have a pvalue under 0.05 that means we have small chances of being wrong is by keeping them.

R-squared value is high (81.7%) and increased a slightly when we removed the parameters.

AIC and BIC are almost zero so we can assume they are low. 

We still have some warnings displayed at the end of the model, especially about multicollinearity,
so we should check our assumptions and transform our data if needed.

"""

## Quick check of OLS assumptions

In [ ]:
# Re-building the model outside of the function 

model = sm.OLS(y,add_constant(X))
model_fit=model.fit()
model_fit.save('../models/fitted_model_1.pickle')
model_fit.summary()

In [ ]:
# Building the predictions

y_pred=model_fit.predict(add_constant(X))
y_pred

In [ ]:
# Checking if predictions seems to be linear
plt.scatter(y,y_pred)
plt.title('Linearity of predictions')
plt.show()

In [ ]:
# Checking residuals
(y-y_pred).mean()

In [ ]:
# Checking how residuals are displayed
plt.plot(y-y_pred)
plt.title('Residuals Variance')
plt.show()

In [ ]:
resid=y-y_pred
sns.distplot(resid)
plt.title('Distribution of residuals')
plt.show()

In [ ]:
"""
Summary of first checking: 

The predictions seems to be linear even if we can still see some outliers at the begining. 

The average of residuals value is almost 0 so our errors seems to be minimized. 
In the meantime, we can see the noise is not so regular and we can clearly identify outliers. 

Then, we can see residuals look like normally distributed but we should still confirm that with the hypothesis.

Finally, we should check the assumptions mathematically. 

"""

## Manual check of assumptions

### 1. Multicollinearity

Checking Variance Inflation Factor for parameters. The threshold is 10, if the parameters is above 10 we should drop the parameter.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF

def drop_check_vif(column, X):
    if column:
        X=X.drop(column, axis=1)
    vifs=pd.Series([VIF(X.values,i) for i in range(X.shape[1])],index=X.columns)
    display(vifs[vifs>10])
    return X

In [ ]:
X = drop_check_vif('',X)

## Iteration on model-1

Following the global check of assumptions on model-1 ([assumptions-model-1.ipynb](assumptions-model-1.ipynb)), it doesn't meet the assumptions so we should iterate to create a new model and check again the assumptions. 

Possible iterations:
* removing infant_deaths and under-five_deaths params because they are the most correlated to others
* removing polio, diphtheria and schooling params if they are still above 10

In [ ]:
X2 = X.copy()

In [ ]:
dropped_cols=[]

In [ ]:
dropped_cols.append('schooling')
dropped_cols[-1]

In [ ]:
X2 = drop_check_vif(dropped_cols[-1],X2)

In [ ]:
# Re-building the model outside of the function

model = sm.OLS(y,add_constant(X2))
model_fit=model.fit()
model_fit.summary()

In [ ]:
# Saving as model-2

model_fit.save('../models/fitted_model_2.pickle')

## Iteration on model-2
By checking the assumptions on model-2 ([assumptions-model-2.ipynb](assumptions-model-2.ipynb)), they are still no met.

Possible iterations:
* find a way to make the measles parameter follow linearity
* perform a non-linear transformation 

In [ ]:
X3 = X2.copy()

In [ ]:
# Creating a list of columns dropped to keep track of their dropping

dropped_cols=[]

In [ ]:
dropped_cols.append('')
dropped_cols[-1]

In [ ]:
# Checking again the model, the R2 value and if parameters are still relevant.

X3=phacking('',X3,y)

In [ ]:
X3.shape

In [ ]:
# Checking the distribution of each parameters except for dummies because it is not relevant

fig,axs=plt.subplots(2,4,figsize=(17,6))

for i in range(X3.shape[1]-2):
    ax = axs[i//4,i%4]
    sns.distplot(X3.iloc[:,i],ax=ax);

In [ ]:
# Performing a non-linear transformation through standardization of data except for dummies
fig,axs=plt.subplots(2,4,figsize=(17,6))

for i in range(X3.shape[1]-2):
    ax = axs[i//4,i%4]
    sns.distplot(((X3.iloc[:,i]-X3.iloc[:,i].mean())/X3.iloc[:,i].std()),ax=ax);

In [ ]:
# Performing a logarithmic non-linear transformation except for dummies because it is not relevant

fig,axs=plt.subplots(2,4,figsize=(17,6))

for i in range(X3.shape[1]-2):
    ax = axs[i//4,i%4]
    sns.distplot(((X3.iloc[:,i]+0.000000001).apply(np.log)),ax=ax);

fig.suptitle('Logarithmic transformation')
plt.show()

In [ ]:
# Re-building the model outside of the function

model = sm.OLS(y,add_constant(X3))
model_fit=model.fit()
model_fit.summary()

In [ ]:
# Saving as model-3
model_fit.save('../models/fitted_model_3.pickle')

In [ ]:
"""
Conclusion on the thrid iteration: 

We can see that data is not really normmaly distributed and a basic non-linear transformation is not enough
because we can see a lot a data with around 0 value. 

=> We should rather split our data into set of common behaviours, 
such as status of countries (Developed, Developing, Lead Developed).

After creating a model for each country status, we check if:
- meales is still violating linearity
- autocorrelation is still present

"""

## Build a model for each status of country

### 1. Model for Developed countries

In [ ]:
df_dev = pd.read_csv('../data/expectancy_dev.csv')
df_dev.head()

In [ ]:
dev_y = df_dev.life_expectancy
dev_X = df_dev.drop(columns=['life_expectancy','status'])

In [ ]:
dev_model = sm.OLS(dev_y,add_constant(dev_X))
dev_model_fit = dev_model.fit()
dev_model_fit.summary()

In [ ]:
dropped_cols = []

In [ ]:
dropped_cols.append('measles')
dropped_cols[-1]

In [ ]:
dev_X = phacking(dropped_cols[-1],dev_X,dev_y)

In [ ]:
# Rebuilding the model outside of the function and saving for assumptions check

dev_model = sm.OLS(dev_y,add_constant(dev_X))
dev_model_fit = dev_model.fit()
dev_model_fit.save('../models/dev-fitted-model-1.pickle')

### 1.1 Iteration on dev-model-1
By checking the assumptions on dev-model-1 ([assumptions-dev-model-1.ipynb](assumptions-dev-model-1.ipynb)), they are still not met.

Possible iterations:
* removing hiv/aids, infant_deaths and under-five_deaths params because they have VIF > 10
* find a way to make the bmi, total_expenditure, diphtheria parameters follow linearity
* correct the autocorrelation 

In [ ]:
dev_X2 = dev_X.copy()
print(dev_X2.shape)
dev_X2.head()

In [ ]:
# Creating list of columns dropped to keep track
dropped_cols = []

In [ ]:
dropped_cols.append('diphtheria')
dropped_cols[-1]

In [ ]:
dev_X2 = drop_check_vif(dropped_cols[-1],dev_X2)

In [ ]:
dev_X2 = phacking('under-five_deaths',dev_X2,dev_y)

In [ ]:
model = sm.OLS(dev_y,add_constant(dev_X2))
model_fit=model.fit()
model_fit.summary()

## Conclusion 

We ended up with a better performance model because AIC is much lower than the model built with all countries. 

But we still have the same issues on assumptions about autocorrelation and exegoneity of residuals (normal fistribution of them).

To fix the first assumptions, we should try to apply Generalized Least Squares (GLS) model which better fit with dataset having a lot of correlation as we actually have in our dataset. 